# CHALLENGE - PROVI

## Hacker News (https://news.ycombinator.com/news)

### THE CHALLENGE IS DEVELOP A SCRIP TO MAKE A WEB SCRAPING

    . Hacker News is a social news website focusing on computer science and entrepreneurship.
    

## LIBRARIES 

In [2]:
import pandas as pd
import requests as req
import re
import time
import json
from bs4 import BeautifulSoup

## Web Scrapping

In [208]:
visited_pages = []
base_url = 'https://news.ycombinator.com/'
next_page = 'https://news.ycombinator.com/ask'

number_question = []
title_question = []
link_question = []
points_question = []
author = []
number_comments = []

while next_page not in visited_pages:
    r = req.get(next_page)
    soup = BeautifulSoup(r.text)
    body_pages = soup.select(".itemlist")
    
    for body_page in body_pages:
    
        num_question = body_page.findAll('span', {'class':'rank'})
        for nq in num_question:
            number_question.append(nq.get_text().strip())
    
        tl_question = body_page.findAll('a', {'class':'storylink'})
        for t_q in tl_question:
            title_question.append(t_q.get_text().strip())
            
        lk_question = body_page.findAll('a', {'class':'storylink'})
        for lk_q in lk_question:
            link_question.append('https://news.ycombinator.com/{}'.format(lk_q['href'].strip()))
          
        
        pt_question = body_page.findAll('span', {'class':'score'})
        for pt_q in pt_question:
            points_question.append(pt_q.get_text().strip())
         
        
        author_question = body_page.findAll('a', {'class':'hnuser'})
        for a_q in author_question:
            author.append(a_q.get_text().strip())
           
        
        numb_comments = body_page.findAll('td', {'class':'subtext'})
        for n_c in numb_comments:
            a_comments = n_c.findAll('a')  
            for a_c in a_comments:
                number_comments.append(re.findall(r"^(\d+\s[a-zA-Z]+?|discuss)$", a_c.get_text().strip()))
        
        number_comments = list(filter(None, number_comments))
        
        
    next_page_link = soup.find('a', {'class':'morelink', 'rel': 'next'})
    visited_pages.append(next_page)
    if next_page_link is not None:
        next_page = 'https://news.ycombinator.com/' + next_page_link['href']
        print(next_page)
    else:
        break
        
    time.sleep(3)
    
    
        

https://news.ycombinator.com/ask?p=2
https://news.ycombinator.com/ask?p=3


In [209]:
hn_df = pd.DataFrame({
    'number_question' : number_question,
    'title_question' : title_question,
    'link_question': link_question,
    'points_question': points_question,
    'author': author,
    'number_comments': number_comments
})

In [210]:
hn_df.head()

,number_question,title_question,link_question,points_question,author,number_comments
0,1.,Ask HN: Do you use inheritance when programming?,https://news.ycombinator.com/item?id=23825310,11 points,jameskerr,[11 comments]
1,2.,Ask HN: How do you document long term products,https://news.ycombinator.com/item?id=23825252,8 points,lobsang,[discuss]
2,3.,"Ask HN: Advanced Linux users, which distributi...",https://news.ycombinator.com/item?id=23816007,62 points,psxuaw,[141 comments]
3,4.,Ask HN: How do I look you in the eye while loo...,https://news.ycombinator.com/item?id=23823397,4 points,nscalf,[6 comments]
4,5.,What is the future of short-term travel?,https://news.ycombinator.com/item?id=23823054,5 points,soumyabruin,[2 comments]


# Data Cleaning

In [211]:
hn_df['number_question'] = hn_df['number_question'].str.replace('.', '')
hn_df['points_question'] = hn_df['points_question'].str.replace('points', '')

In [212]:
hn_df['number_comments']

0      [11 comments]
1          [discuss]
2     [141 comments]
3       [6 comments]
4       [2 comments]
           ...      
73      [7 comments]
74         [discuss]
75         [discuss]
76       [1 comment]
77         [discuss]
Name: number_comments, Length: 78, dtype: object

In [213]:
number = hn_df['number_comments'].astype(str)

hn_df['number_comments'] = number.str.extract('(\d+)')

In [214]:
hn_df['number_comments'] = hn_df['number_comments'].fillna(0)

In [215]:
hn_df['number_comments'] = pd.to_numeric(hn_df['number_comments'])
hn_df['number_question'] = pd.to_numeric(hn_df['number_question'])

In [216]:
hn_df.head(40)

,number_question,title_question,link_question,points_question,author,number_comments
0,1,Ask HN: Do you use inheritance when programming?,https://news.ycombinator.com/item?id=23825310,11,jameskerr,11
1,2,Ask HN: How do you document long term products,https://news.ycombinator.com/item?id=23825252,8,lobsang,0
2,3,"Ask HN: Advanced Linux users, which distributi...",https://news.ycombinator.com/item?id=23816007,62,psxuaw,141
3,4,Ask HN: How do I look you in the eye while loo...,https://news.ycombinator.com/item?id=23823397,4,nscalf,6
4,5,What is the future of short-term travel?,https://news.ycombinator.com/item?id=23823054,5,soumyabruin,2
5,6,Why can Apple out CPU Intel?,https://news.ycombinator.com/item?id=23815712,41,raasdnil,38
6,7,Femto Computing,https://news.ycombinator.com/item?id=23822777,2,AAMOUSA0519,2
7,8,Ask HN: How does Superhuman Mail tracks every ...,https://news.ycombinator.com/item?id=23822711,2,dtarik,3
8,9,Ask HN: Multi-tenancy in software vs. infrastr...,https://news.ycombinator.com/item?id=23820461,8,slayerjain,12
9,10,CMS that dynamically changes your website base...,https://news.ycombinator.com/item?id=23821587,2,matteomosca,3


In [217]:
len(hn_df['number_question'])

78

# Data Analysis

In [218]:
## HOW MANY POST ARE AVAILABLE IN THE /ASK SECTION
print(len(hn_df['number_question']))

78


### How many  post are available in the /ask section?

       R. The result so far is 78, and the reason for that I know that it is because one of the parts of the website have the number of the question and I get all question in the /ask section. So because of that the last number of the quest shows how many post are available.

In [220]:
hn_df_rank = hn_df
hn_df_rank['rank_comments'] = hn_df_rank['number_comments'].rank(ascending = 0)

hn_df_rank = hn_df_rank.set_index('rank_comments')

In [340]:
hn_df_rank = hn_df_rank.sort_index()
hn_df_rank.head(10)

,number_question,title_question,link_question,points_question,author,number_comments
0,11,Ask HN: What's the worst piece of software you...,https://news.ycombinator.com/item?id=23803539,510,guu,1459
1,3,"Ask HN: Advanced Linux users, which distributi...",https://news.ycombinator.com/item?id=23816007,62,psxuaw,141
2,16,Ask HN: How can I quickly trim my AWS bill?,https://news.ycombinator.com/item?id=23798347,149,danicgross,130
3,39,Tell HN: 6.3% of HN top submissions in plain H...,https://news.ycombinator.com/item?id=23802521,100,oefrha,73
4,38,Ask HN: Best resources to learn about stock tr...,https://news.ycombinator.com/item?id=23796028,68,vkbm,57
5,66,Ask HN: What's the best piece of software you ...,https://news.ycombinator.com/item?id=23806012,31,thdrdt,51
6,17,Ask HN: How do you learn new libraries without...,https://news.ycombinator.com/item?id=23800590,48,vedant_shety,51
7,49,Ask HN: How many hours do you work?,https://news.ycombinator.com/item?id=23807304,33,xupybd,48
8,50,Ask HN: Concepts the modern tech industry fals...,https://news.ycombinator.com/item?id=23814645,45,Ozzie_osman,48
9,29,Ask HN: 9-5 Burnout?,https://news.ycombinator.com/item?id=23822895,32,notoriousarun,43


In [242]:
hn_df_rank = hn_df_rank.rename_axis(None)

In [249]:
hn_df_rank = hn_df_rank.reset_index(drop=True)

In [252]:
hn_df_rank['link_question'][0]

'https://news.ycombinator.com/item?id=23803539'

In [339]:
hn_df_rank.head()

,number_question,title_question,link_question,points_question,author,number_comments
0,11,Ask HN: What's the worst piece of software you...,https://news.ycombinator.com/item?id=23803539,510,guu,1459
1,3,"Ask HN: Advanced Linux users, which distributi...",https://news.ycombinator.com/item?id=23816007,62,psxuaw,141
2,16,Ask HN: How can I quickly trim my AWS bill?,https://news.ycombinator.com/item?id=23798347,149,danicgross,130
3,39,Tell HN: 6.3% of HN top submissions in plain H...,https://news.ycombinator.com/item?id=23802521,100,oefrha,73
4,38,Ask HN: Best resources to learn about stock tr...,https://news.ycombinator.com/item?id=23796028,68,vkbm,57


## Data Analysis - Count the most frequent words and respective number of frequency

In [270]:
from collections import Counter

In [300]:
"""
This function is responsable for get a url and return the word and the frequency 
    that appears in the specif tag of website
"""
def frequency_words(url):
    data = []
    r = req.get(url)
    soup = BeautifulSoup(r.text)
    body_comments = soup.select(".comment-tree")
    
    for body_comment in body_comments:
        bc_aux = body_comment.findAll('div', {'class':'comment'})
        for t in bc_aux:
            data.append(t.get_text())

            
        separator = ' '
        data_aux = separator.join(data)
        
        split_it = data_aux.split()
        count_var = Counter(split_it)
        most_occur = count_var.most_common(3)
        
        return most_occur
        

In [326]:
nb_question = []
title_question = []
url = []
word0_frequency = []
quant0_frequency = []
word1_frequency = []
quant1_frequency = []
word2_frequency = []
quant2_frequency = []


for i in range(10):
    nb_question.append(hn_df_rank['number_question'][i])
    title_question.append(hn_df_rank['title_question'][i])
    url.append(hn_df_rank['link_question'][i])
    
    frequency_aux = frequency_words(hn_df_rank['link_question'][i])
    for i in range(3):
        globals()["word"+str(i)+"_frequency"].append(frequency_aux[i][0])
        globals()["quant"+str(i)+"_frequency"].append(frequency_aux[i][1])


In [327]:
df_common_word = pd.DataFrame({
    'number_question': nb_question,
    'title_question' : title_question,
    'url' : url,
    'word_frequency_0' : word0_frequency,
    'quant_frequency_0' : quant0_frequency,
    'word_frequency_1' : word1_frequency,
    'quant_frequency_1' : quant1_frequency,
    'word_frequency_2' : word2_frequency,
    'quant_frequency_2' : quant2_frequency
})

In [328]:
df_common_word.head(10)

,number_question,title_question,url,word_frequency_0,quant_frequency_0,word_frequency_1,quant_frequency_1,word_frequency_2,quant_frequency_2
0,11,Ask HN: What's the worst piece of software you...,https://news.ycombinator.com/item?id=23803539,the,602,to,561,a,467
1,3,"Ask HN: Advanced Linux users, which distributi...",https://news.ycombinator.com/item?id=23816007,I,367,the,319,and,317
2,16,Ask HN: How can I quickly trim my AWS bill?,https://news.ycombinator.com/item?id=23798347,to,277,the,247,a,216
3,39,Tell HN: 6.3% of HN top submissions in plain H...,https://news.ycombinator.com/item?id=23802521,the,130,to,127,a,86
4,38,Ask HN: Best resources to learn about stock tr...,https://news.ycombinator.com/item?id=23796028,to,103,the,89,a,74
5,66,Ask HN: What's the best piece of software you ...,https://news.ycombinator.com/item?id=23806012,reply,51,the,40,and,36
6,17,Ask HN: How do you learn new libraries without...,https://news.ycombinator.com/item?id=23800590,the,160,to,135,you,79
7,49,Ask HN: How many hours do you work?,https://news.ycombinator.com/item?id=23807304,I,106,a,97,to,86
8,50,Ask HN: Concepts the modern tech industry fals...,https://news.ycombinator.com/item?id=23814645,the,63,reply,48,a,40
9,29,Ask HN: 9-5 Burnout?,https://news.ycombinator.com/item?id=23822895,a,107,to,94,I,89


## What other data source or news website could be used to prove a strong correlation between the posts with more comments?

### For this particular question, it is possible to find some good examples like:

    . StackOverflow (https://stackoverflow.com/)
    . Reddit (https://www.reddit.com/)
    . Quora (https://pt.quora.com/)
    . Steemit (https://steemit.com/)
    . Slashdot (https://slashdot.org/)
    . Lobsters (https://lobste.rs/)
    . Indie Hackers (https://www.indiehackers.com/)

## There is some correlation between the content of Hacker News website with others site of the same genre in the web?

### <font color='red'> Yes. For example:

    . Indie Hackers
    . Tildes (www.tildes.net)
    . Quora
    . Reddit
    . StackOverflow 
    
### All of these website have something in comum, that is the possibility of any person can share their story, their project, some news. What the community can come together to share their experiences, give and receive feedback, and rely on each other for support.

### <font color='red'>Based on the user who post a question or comment, there is any correlation between the available information of an user vs the post / comment he did? Is it possible to find any pattern between those who post new questions or comment? </font>



     R. What I know so far, that hack news have low information about the users, they just have their post and what they commented. Because of that, to finding some correlation I think the best way is find the key words that this person mostly post or comment, with that is possible to find some pattern.

# Save the data in a CSV


In [338]:
## SAVE TO CSV THE DATA WITHOUT MAKE A RANKING OF THE FREQUENCY OF WORDS

hn_df.to_csv(r'./hacker_news.csv', index = False, header=True)

## NEXT CSV WITH THE RANKING OF THE FREQUENCY OF WORDS

hn_df_rank.to_csv(r'./hacker_news_rank_words.csv', index = False, header=True)


# UnitTest for web scraping


In [330]:
import unittest
from urllib.request import urlopen

class Test(unittest.TestCase):
    bs = None
    
    def setUpClass():
        url = 'https://news.ycombinator.com/ask'
        Test.bs = BeautifulSoup(urlopen(url), 'html.parser')
    def test_textRank(self):
        textRank = Test.bs.find('span', {'class':'rank'})
        self.assertEqual('1.', textRank.text)
    def test_rankExists(self):
        rank = Test.bs.findAll('span', {'class':'rank'})
        self.assertIsNotNone(rank)
    def test_titleQuestionExists(self):
        titleQuestion = Test.bs.findAll('a', {'class':'storylink'})
        self.assertIsNotNone(titleQuestion)
    def test_pointsIsRight(self):
        pt_question = Test.bs.find('span', {'class':'score'})
        self.assertNotEqual('7', pt_question.text)
        
        
if __name__ == 'main':
    unittest.main()


In [331]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_pointsIsRight (__main__.Test) ... ok
test_rankExists (__main__.Test) ... ok
test_textRank (__main__.Test) ... ok
test_titleQuestionExists (__main__.Test) ... ok

----------------------------------------------------------------------
Ran 4 tests in 1.620s

OK
